In [4]:
import os
os.chdir( "/home/lucasfuzato/CODING/WINE" )

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

from src.aux_fun import *

tests_df = pd.DataFrame( columns = [ "tree_num" , "num_features" ] )

tree_size = [ 5 , 10 , 50 , 100 ]
feature_select = [ "log2" , "sqrt"]
for i , ( tree_num , num_features ) in enumerate( product( tree_size , feature_select ) ):
    tests_df.loc[ i ] = pd.Series( [ tree_num , num_features ] , [ "tree_num" , "num_features" ] )
print( tests_df ) 

  tree_num num_features
0        5         log2
1        5         sqrt
2       10         log2
3       10         sqrt
4       50         log2
5       50         sqrt
6      100         log2
7      100         sqrt


In [6]:
from sklearn.ensemble import RandomForestClassifier as rd_frst
seed_val = 47

depth , algo = 20 , "gini"

X , y , X_2d , columns = class_get_data()

scores = [ "f1_score" , "precision" , "recall" ]
metrics = [ "mean" , "std" ]
data = { tup:list() for tup in product( scores , metrics ) }

for simu_id , ( tree_num , num_features ) in enumerate( product( tree_size , feature_select ) ):

    lst = []
    fold_iter = get_prepared_data( X , y , seed = seed_val , smote = False )
    for ( X_train , y_train ) , ( X_test , y_test ) in fold_iter:

        forest = rd_frst( n_estimators = tree_num , max_features = num_features, criterion = algo , max_depth = depth ,  )
        forest.fit( X_train , y_train )

        ts_cls = forest.predict( X_test )
        ts_f1 , prec , recall = bin_f1_score( ts_cls , y_test )

        lst.append( ( ts_f1 , prec , recall ) )
    
    arr = np.array( lst )
    means = arr.mean( axis = 0 )
    data[ ( "f1_score", "mean" ) ].append( means[ 0 ] )
    data[ ( "precision", "mean" ) ].append( means[ 1 ] )
    data[ ( "recall", "mean" ) ].append( means[ 2 ] )

    stds  = arr.std( axis = 0 )
    data[ ( "f1_score", "std" ) ].append( stds[ 0 ] )
    data[ ( "precision", "std" ) ].append( stds[ 1 ] )
    data[ ( "recall", "std" ) ].append( stds[ 2 ] )

print( pd.DataFrame( data ) )
print()

   f1_score           precision              recall          
       mean       std      mean       std      mean       std
0  0.652205  0.033378  0.709810  0.041409  0.605290  0.044217
1  0.636421  0.031501  0.707430  0.036697  0.580297  0.041695
2  0.648361  0.023065  0.783072  0.034006  0.554454  0.031111
3  0.643606  0.035503  0.773820  0.034032  0.552947  0.047109
4  0.682448  0.025157  0.813605  0.035683  0.588903  0.031987
5  0.682671  0.035616  0.816231  0.035150  0.587383  0.039524
6  0.695868  0.022892  0.827304  0.036057  0.601415  0.026632
7  0.695654  0.026537  0.829482  0.038960  0.599883  0.028785

